# create keras model with 1 hidden layer of same size as input

In [1]:
# TOFIX 
# (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
# Solution in: https://github.com/tensorflow/tensorflow/issues/24828
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
import tensorflow.compat.v1 as tf

In [3]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)

In [4]:
from tensorflow import keras

In [5]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=10, kernel_size=1, input_shape=(1,1,10))
])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# compile and show model summary (Do I need to compile model?)

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.cosine,
    metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1, 1, 10)          110       
Total params: 110
Trainable params: 110
Non-trainable params: 0
_________________________________________________________________


# predict on some inputs

In [7]:
import numpy as np
x = np.ones((2,10), dtype=np.float32).reshape(2,1,1,10)
model.predict(x=x)

array([[[[-2.3906312 , -0.44901997, -0.82550603,  0.48844722,
          -0.972329  , -0.8711493 , -0.659256  , -1.0125457 ,
           0.68088347,  1.9050514 ]]],


       [[[-2.3906312 , -0.44901997, -0.82550603,  0.48844722,
          -0.972329  , -0.8711493 , -0.659256  , -1.0125457 ,
           0.68088347,  1.9050514 ]]]], dtype=float32)

# get model's layer's weights

In [8]:
model.layers[0].get_weights()

[array([[[[-0.15053287, -0.26998153,  0.39287734,  0.18240607,
           -0.42053258, -0.25637227, -0.19844729, -0.11023614,
            0.52403355,  0.51810074],
          [-0.4331238 ,  0.16207117, -0.3886429 ,  0.0289607 ,
           -0.31068376, -0.5258798 , -0.05812842, -0.2965964 ,
           -0.48082495,  0.37170327],
          [-0.5220739 ,  0.00224346, -0.37051976, -0.06845486,
           -0.52132356, -0.10841158, -0.52159244,  0.10140163,
           -0.17957065,  0.23109084],
          [-0.42687938, -0.19117123, -0.03836012,  0.17736334,
            0.42864662, -0.48629773,  0.01950884,  0.11742014,
           -0.09644026,  0.30683887],
          [-0.28665844,  0.07926214,  0.34955633,  0.08781576,
            0.30137497,  0.15698063, -0.14481813,  0.03726763,
            0.10285336,  0.10013354],
          [-0.41407686,  0.22930467,  0.17208421, -0.5359557 ,
           -0.33217058,  0.05746293,  0.1183666 , -0.01634091,
           -0.27730408,  0.02567399],
          [ 0.47

In [9]:
model.layers[0].get_weights()[0].shape

(1, 1, 10, 10)

In [10]:
model.layers[0].get_weights()[1].shape

(10,)

# set weights to get identical model

In [11]:
model.layers[0].set_weights([
    np.identity(10, dtype=np.float32).reshape(1, 1, 10, 10),
    np.full(10, 0.0001, dtype=np.float32)
])

# check weights agian

In [12]:
model.layers[0].get_weights()

[array([[[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]]], dtype=float32),
 array([1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
        1.e-04, 1.e-04], dtype=float32)]

# check prediction

In [13]:
model.predict(x)

array([[[[1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001,
          1.0001, 1.0001, 1.0001]]],


       [[[1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001,
          1.0001, 1.0001, 1.0001]]]], dtype=float32)

In [14]:
x

array([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]],


       [[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]]], dtype=float32)

# save model

In [15]:
tf_session = keras.backend.get_session()

# write out tensorflow checkpoint & meta graph
saver = tf.train.Saver()

In [16]:
save_path = saver.save(tf_session,"conv_identical/conv_identical.ckpt")

# save model as an HDF5 file

In [17]:
model.save('conv_identical/conv_identical.h5')

# freeze saved model

In [18]:
model.outputs

[<tf.Tensor 'conv2d/BiasAdd:0' shape=(?, 1, 1, 10) dtype=float32>]

In [19]:
!CUDA_VISIBLE_DEVICES=-1 python freeze_graph.py \
    --input_meta_graph conv_identical/conv_identical.ckpt.meta \
    --input_checkpoint conv_identical/conv_identical.ckpt \
    --output_graph conv_identical/conv_identical.pb \
    --output_node_names=conv2d/BiasAdd \
    --input_binary=true

Loaded meta graph file 'conv_identical/conv_identical.ckpt.meta
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W1203 08:57:45.639789 140044863121216 deprecation.py:323] From freeze_graph.py:129: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2020-12-03 08:57:45.642804: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-03 08:57:45.672631: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-03 08:57:45.672733: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: giangblackk-HP
2020-12-03 08:57:45.672763: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname